In [2]:
import pandas as pd

In [7]:
# Create sample data with 5-minute intervals starting at 9:30 AM
index = pd.date_range(start='2024-03-20 09:30:00', periods=5, freq='5min')

df = pd.DataFrame({
    'volume': [1000, 1500, 800, 2000, 1200],
    'vwap': [150.25, 150.50, 150.40, 150.75, 150.60],
    'transactions': [50, 75, 40, 100, 60],
    'close': [150.30, 150.55, 150.35, 150.80, 150.65],
    'open': [150.20, 150.45, 150.45, 150.70, 150.55],
    'high': [150.35, 150.60, 150.50, 150.85, 150.70],
    'low': [150.15, 150.40, 150.30, 150.65, 150.50],
    'avg_size': [20, 20, 20, 20, 20],
    'ticker': ['AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL']
}, index=index)

df


,volume,vwap,transactions,close,open,high,low,avg_size,ticker
2024-03-20 09:30:00,1000,150.25,50,150.30,150.20,150.35,150.15,20,AAPL
2024-03-20 09:35:00,1500,150.50,75,150.55,150.45,150.60,150.40,20,AAPL
2024-03-20 09:40:00,800,150.40,40,150.35,150.45,150.50,150.30,20,AAPL
2024-03-20 09:45:00,2000,150.75,100,150.80,150.70,150.85,150.65,20,AAPL
2024-03-20 09:50:00,1200,150.60,60,150.65,150.55,150.70,150.50,20,AAPL


In [4]:
# Null out the last row
df.iloc[-1] = None
df


,volume,vwap,transactions,close,open,high,low,avg_size,ticker
2024-03-20 09:30:00,1000.0,150.25,50.0,150.30,150.20,150.35,150.15,20.0,AAPL
2024-03-20 09:35:00,1500.0,150.50,75.0,150.55,150.45,150.60,150.40,20.0,AAPL
2024-03-20 09:40:00,800.0,150.40,40.0,150.35,150.45,150.50,150.30,20.0,AAPL
2024-03-20 09:45:00,2000.0,150.75,100.0,150.80,150.70,150.85,150.65,20.0,AAPL
2024-03-20 09:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [5]:
from phantom_core.ohlcv import fill_ohlcv

In [8]:
fill_ohlcv(df)

,volume,vwap,transactions,close,open,high,low,avg_size,ticker
2024-03-20 09:30:00,1000,150.25,50,150.30,150.20,150.35,150.15,20,AAPL
2024-03-20 09:35:00,1500,150.50,75,150.55,150.45,150.60,150.40,20,AAPL
2024-03-20 09:40:00,800,150.40,40,150.35,150.45,150.50,150.30,20,AAPL
2024-03-20 09:45:00,2000,150.75,100,150.80,150.70,150.85,150.65,20,AAPL
2024-03-20 09:50:00,1200,150.60,60,150.65,150.55,150.70,150.50,20,AAPL
